# Introduction

This notebook contains the scripts needed to run the nested cross-validation (nCV) mentioned in the method of part 2 of the project. The notebook contains a variety of functions needed by the nCV from data cleaning to machine learning algorithm (MLA) hyperparameter tuning and plotting the receiver operator characteristic plot of the outer runs. The functions in the notebook can be grouped into the following 5 categories and consequent 5 main steps:

Step 1: The libraries needed to run the LSTM and other dataset-altering functions are imported alongside the datasets containing the  sMRI volume measurements, the sMRI thickness measurements, the dMRI, the clinical, and the demographic data. The dMRI data is however only used if the first 2 sessions (Y00 and Y05) are intended to be used for training and evaluating the model as it does not currently have any available data for the third session (Y10).

Step 2: The first 5 functions: get_dMRI, clean_sMRI_vol, PRESGENE_ID_for_thickness, clean_sMRI_thickness, and make_base_df are used to create the base dataset from the input datasets the steps taken to clean these including removing rows or columns with more than 90% of missing values, isolate features of interest and participants based on the target variable and combining the datasets with the desired number of sessions. Once the base dataset is obtained, the missing values are filled with the Time + Type imputation method using the functions split_df_by_time and impute_df. As in part 1 of the project, the overall class imbalance is handled using Synthetic Minority Over-sampling Technique (SMOTE) by reshaping the dataset to 1 row per subject in the function merge_time_by_rows, applying the SMOTE algorithm to generate new participant data based on HC / MS phenotype in the function class_rebalancing_SMOTE, and then reformat the dataset back to n rows per participant for the n sessions in the function reformat_SMOTE_dfs. Depending on the desired transformation method (None or UMAP), the function apply_unique_UMAP is used to dimensionally reduce the dataset to 2 Features with the hyperparameters (hps) of the algorithm (perplexity, minimum distance, and metric) tuned with the mode of the hps obtained in part 1 of the project. The final step in preparing the dataset is feature engineering if the desired target variable is bi_EDSS, tri_EDSS, worsening_EDSS, or RR_to_SP, then the feature is created in this function. Otherwise, the function ensures that the target variable has the correct format. The new feature is then added to the dataframe or the existing column is updated in the dataframe.

Step 3: Encompasses all of the functions needed for the LSTM to run successfully. The dataset is reformated to a 1 row per participant format in prep_LSTM_data, then either the function rdm_oversampling is used to apply random oversampling to the dataset when the target variable is categorical or bootstrapping is used to increase the sample size with duplicates. The tune_lstm is used to tune the LSTM hps in the inner loop of the nCV and either the lstm_regressor_outer or the lstm_classifier_outer functions are used to build the LSTM in the function run_lstm with the hps found during tuning in the outer loop of the nCV. The function evaluate_score is used to compare the loss functions across the inner and outer runs of the LSTM and update the hps or model for the corresponding best score. Finally, the function AUCROC is used to obtain the AUCROC plot during the outer runs of the nCV.

Step 4: Contains the function for the nCV and groups the previously mentioned functions into a nCV pipeline in the function CV_pipeline. After the creation of the base dataset with the functions mentioned in step 2, get_unique_subs is used to obtain a list of the participants representative of the target variable that stratifiedKfold can use. For example, continuous target variables (e.g.: average cogntion) are binned into 3 classes so that each range of values is evenly represented in each fold. These alterations are temporary and only used for stratifying the dataset when obtaining the folds. Following imputation and SMOTE, a version of the dataset with all combined sessions is obtained with the function merge_time_imputed so that it i. may also be used as a training dataset and ii. be used for feature engineering in the overarching function apply_transform_labeled which also performs data transformation on each of the train and test dataset for the given nCV fold. As demonstrated in Figure 2 of the paper, the nCV pipeline repeats the previous steps as it iterates through the inner folds of the outer folds. In the end, a dictionary of the inner and outer evaluation scores and best hps are returned, in addition to AUCROC plots being saved (for categorical target variables), and the best performing-model.

Step 5: This final step provides a few cells to run the nCV with the possibility to choose the parameter values: transformation (RAW/UMAP), number of sessions to include in the training data (n_ses), session to use as test set (n_test), number of outer nCV runs (n_outer), and number of inner nCV runs (n_inner). A list of the target/dependent variable ('EDSS', 'MS', 'MStype', 'BL_Avg_cognition', 'HC_CI_CP','bi_EDSS', 'tri_EDSS', 'worsening_EDSS', 'RR_to_SP'), is also provided and the cell is set up such that we can iterate through the list of target variables with the nCV being repeated for each of the variables and the outputs of the function CV_pipeline being saved and added to a master dictionary for easy access to the combined target variables for analysis.

This pipeline demonstrates how the results for part 2 of the project were obtained. The LSTM was isolated from the other MLAs as it requires a different version of certain libraries as explained in the LSTM subsection of the methodology.

# Step 1: Import Libraries & Datasets

In [ ]:
# Base
import re
import os
import pickle
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import StratifiedKFold

# Dimensionalty Reduction
from umap import UMAP
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Machine learning algorithms general
import random
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error, r2_score, explained_variance_score

# machine learning LSTM specific
import keras
import keras_tuner as kt
from keras_tuner import Objective
from tensorflow.keras.models import save_model, model_from_json, Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import RandomSearch
from tensorflow.keras.utils import to_categorical
from keras_tuner.engine.hyperparameters import HyperParameters

In [ ]:
# load the diffusion MRI datasets
dMRI = pd.read_excel('data/Presgene_wholeBL_forFBA_metrics_JHU_forLME_zscores.xlsx')

# Load the structural MRI dataset (volumes)
volume_df = pd.read_excel('updated_data/prograMS_database_v1.9_reduced_kayna.xlsx')

# Create the sublists to group the sMRI (thickness) time points together
Y00, Y05, Y10 = list(), list(), list()

# Load the structural MRI datasets (thickness)
sMRIs_path = 'data/sMRIs'
for filename in os.listdir(sMRIs_path):
    # Read and store each file in its appropriate list
    file_path = os.path.join(sMRIs_path, filename)
    df = pd.read_csv(file_path, sep='\t')
    
    if 'Y00' in filename:
        Y00.append(df)
    elif 'Y05' in filename:
        Y05.append(df)
    elif 'Y10' in filename:
        Y10.append(df)

# Create the final sMRI list with all time points
sMRI_ls = [Y00, Y05, Y10]

# Step 2: Prepare the Data for the LSTM

## Functions for the base dataset

In [ ]:
def get_dMRI(df):
    """
    INPUT: 
    df :(dataframe) dataset with dMRI data
    OUTPUT: dataframe
    DESCRIPTION: Sets up the dMRI dataset for merging with others in the make_base_df function. Isolates the no lession zscores 
    (FD, FDC, and logFC), subject IDs and sessions number.
    """
    # Create a list with all columns begining with FD or Logfc in the df
    all_dMRI_cols = [col for col in df.columns if col.startswith('FD') or col.startswith('Logfc')]
    
    # Keep only columns with _nolesion_zscores
    keep_dMRI_cols = [col for col in all_dMRI_cols if col.endswith('_nolesion_zscores')]

    # Get the columns to remove (not the selected FD/Logfc, PRESGENE)ID and Time) columns
    rm_cols = [col for col in df.columns if col not in ['PRESGENE_ID', 'Time'] + keep_dMRI_cols]

    # Remove the unwanted columns from the df
    output_df = df.drop(columns = rm_cols, axis = 1)

    return output_df

In [ ]:
def clean_sMRI_vol(df):
    """
    INPUT: 
    df :(dataframe) dataset with demographic, clinical and structural MRI dataset.
    OUTPUT: dataframe
    DESCRIPTION: Sets up the sMRI (volumes) dataset for merging with others in the make_base_df function. 
    Retains the clinical, demographic, and sMRI volume features. Removes rows and columns with at least 90% of missing values.
    """
    # Make a copy of the input df to prevent editing the original df
    working_df = df.copy()
    
    #Remove raw crossectional measures
    clin_demo = ['id_presgene','programs_label_code','edss', 'mstype_code', 'subject_type_code',
                 'age',	'sex',	'average_cognition', 'HC_CI_CP']
    
    vol_long_corr = [col for col in working_df.columns if 'Vol_long_all_corrected' in col and 'Mask' not in col and 'BrainSeg' not in col]

    working_df = working_df[[col for col in working_df.columns if col in clin_demo + vol_long_corr + ['lesion_volume_mm3']]]

    # Replace other missing type notation with the most common MStype
    mstype_mode = working_df['mstype_code'].mode(dropna=True)[0]
    working_df['mstype_code'] = working_df['mstype_code'].apply(lambda x: mstype_mode if x not in {1, 2, 3, None} else x)

    # Remove columns filled with at least 90% missing values or zeros:
    nan_prct = working_df.isnull().mean()
    zr_prct = (working_df == 0).mean()
    prct = nan_prct+zr_prct
    nozr_df = working_df.loc[:, prct < 0.8]

    zr_col = prct[prct >= 0.8].index.tolist()
    print(f"Columns: {zr_col} have more than 80% missing data and were removed")

    # Remove rows filled with at leasst 90% missing values or zeros and any of their occurances:
    na_prct_row = nozr_df.isnull().mean(axis=1)
    zr_prct_row = (nozr_df == 0).mean(axis=1)
    prct_row = na_prct_row + zr_prct_row
    zr_row = nozr_df.loc[prct_row >= 0.8, 'id_presgene'].tolist()
    nozr_df =  nozr_df[~nozr_df['id_presgene'].isin(zr_row)]

    print(f"Rows with ID {zr_row} have more than  80% missing data and were removed")

    # Encode the HC_CI_CP to numerical labels
    nozr_df.loc[:, 'HC_CI_CP'] = nozr_df.loc[:, 'HC_CI_CP'].fillna(9999)
    nozr_df.loc[:,'HC_CI_CP'] = nozr_df.loc[:,'HC_CI_CP'].replace({'HC': 0, 'CP': 1, 'CI': 2}).infer_objects(copy=False)
    nozr_df.loc[:, 'HC_CI_CP'] = nozr_df.loc[:,'HC_CI_CP'].replace(9999, np.nan).infer_objects(copy=False)
    
    # Rename columns for clarity
    rename_dict = {'id_presgene':'PRESGENE_ID',
                   'programs_label_code': 'Time',
                   'edss': 'EDSS',
                   'mstype_code':'MStype',
                   'subject_type_code':'MS',
                   'age': 'Age',
                   'sex': 'Sex',
                   'average_cognition':'BL_Avg_cognition'}
                          
    nozr_df = nozr_df.rename(columns = rename_dict)
    
    return nozr_df

In [ ]:
def PRESGENE_ID_for_thickness(df_list):
    """
    INPUT: 
    df_list : (list of dataframes) list containing 2 sMRI (thickness) datasets representing left & right hemisphere measurements
    OUTPUT: 2 dataframes of left and right hemisphere data.
    DESCRIPTION: Reformats the ID column to match that of the PRESGENE_ID, and identifies the left and right hemisphere dataset.
    """
    lh_df, rh_df = None, None

    for df in df_list:
        old_id = df.columns[0]
        id_col = 'PRESGENE_ID'
        # Restructure ID column
        df = df.rename(columns = {df.columns[0] : id_col})
        df[id_col] = df[id_col].astype(str)
        df[id_col] = df[id_col].replace('^sub-', '', regex=True)
        df[id_col] = df[id_col].str[:4]+ '_' + df[id_col].str[4:]

        # Remove the BrainSegVolNotVent & eTIV columns:
        df = df.drop(columns = ['BrainSegVolNotVent', 'eTIV'])

        if 'lh' in old_id:
            lh_df = df
        elif 'rh' in old_id:
            rh_df = df
        else:
            print('Neither left nor right hemisphere files were provided.')

    return lh_df, rh_df

In [ ]:
def clean_sMRI_thickness(df_list, num_ses):
    """
    INPUT:
    df_list : (nested list of dfs) Each session has its own sublist containing the left and right hemisphere sMRI thickness 
    data within the main list.
    num_ses : (integer) number of sessions to include in the merging of the sMRI thickness datasets
    OUTPUT: 1 dataframe
    DESCRIPTION: Sets up the sMRI (thickness) individual datasets (per session and brain hemisphere) for merging with others in the 
    make_base_df function. Uses the parameter num_ses session to combine the left and right hemisphere data up to the desired session
    (baseline = 1, five year follow-up = 2, ten year follow-up = 3).
    """
    output_df = None

    for i in range(0, num_ses):
        # Rename the subjects to match the PRSEGENE structure:
        presgene_lh, presgene_rh = PRESGENE_ID_for_thickness(df_list[i])
        
        # merge lh & rh datasets & add time column
        lh_rh_df = pd.merge(presgene_lh, presgene_rh, on=['PRESGENE_ID'], how='inner')
        lh_rh_df['Time'] = i + 1

        if i == 0: # if baseline (Y00)
            output_df = lh_rh_df
            
        else:
            # Combine the time points vertically, only keeping subjects common to all desired tps. 
            common_ids = pd.merge(output_df[['PRESGENE_ID']], lh_rh_df[['PRESGENE_ID']], on='PRESGENE_ID')
            base_filtered = output_df[output_df['PRESGENE_ID'].isin(common_ids['PRESGENE_ID'])]
            new_filtered = lh_rh_df[lh_rh_df['PRESGENE_ID'].isin(common_ids['PRESGENE_ID'])]

            output_df = pd.concat([base_filtered, new_filtered], ignore_index=True)
        
    return output_df

In [ ]:
def make_base_df(presgene_df, sMRI_vol, sMRI_ls, num_tps, task_ft):
    """
    INPUT:
    dMRI: (dataframe) diffusion MRI dataset.
    sMRI_vol: (dataframe) structural MRI dataset with volumetric, clinical and demographic data.
    sMRI_ls : (nested list of dataframes) sublists containing the thickness structural MRIs at one of the sessions for the left and 
    right hemisphere of the brain
    num_ses: (integer) the number of the highest session desired in the dataset. (1-3)
    task_ft : (string) current target variable name, determines if base is pwMS only dataset or HC + pwMS dataset
    OUTPUT: 1 dataframe
    DESCRIPTION: Cleans and merges the sMRI thickness, sMRI volumes, and dMRI (if num_tps < 3) datasets up to the desired session number 
    for the current target variable
    """
    # Clean the structural MRI data
    vol_df = clean_sMRI_vol(sMRI_vol)
    thickness_df = clean_sMRI_thickness(sMRI_ls, num_tps)

    if isinstance(presgene_df, pd.DataFrame) and num_tps < 3: 
        dMRI_df = get_dMRI(presgene_df)
        # Merge the dMRI & volume sMRI measures:
        vol_df = pd.merge(vol_df, dMRI_df, on=['PRESGENE_ID', 'Time'])

    # Find their common Subjects & corresponding tps
    common_ids = pd.merge(vol_df[['PRESGENE_ID', 'Time']], thickness_df[['PRESGENE_ID', 'Time']], on=['PRESGENE_ID', 'Time'])
    
    # Filter the data frames to include only common subjects
    pres_filtered = vol_df[vol_df[['PRESGENE_ID', 'Time']].apply(tuple, 1).isin(common_ids.apply(tuple, 1))]
    dMRI_filtered = thickness_df[thickness_df[['PRESGENE_ID', 'Time']].apply(tuple, 1).isin(common_ids.apply(tuple, 1))]

    # Combine the presgene & dMRI datasets
    output_df = pd.merge(pres_filtered, dMRI_filtered, on=['PRESGENE_ID','Time'], how = 'inner')

    # Remove the column of noninterest based on the wanted dataset
    output_df = output_df.drop(columns = ['lesion_volume_mm3'])

    # Remove HC if the task is for pwMS only
    if task_ft in ['MStype', 'bi_EDSS', 'tri_EDSS', 'worsening_EDSS', 'EDSS', 'RR_to_SP']:
        output_df = output_df[output_df['MS'] == 1]

    print(f'Merging completed, base dataset for {task_ft} up to time point {output_df["Time"].max()} is ready!')
    return output_df

## Functions for dataset imputation

In [ ]:
def split_df_by_time(df):
    """
    INPUT: 
    df : (dataframe) dataset with data of 2 or more sessions
    OUTPUT: list of dataframes eaach with data for a unique session
    DESCRIPTION: splits the input dataset into unique sessions according to its max session
    """
    # Find highest session number and initialise output variables
    max_tp = max(df['Time'].value_counts().index)
    output_list = []
    
    # Get baseline
    t1_df = df[df['Time'] == 1]
    output_list.append(t1_df)
    
    # Get session Y05 & remove columns not measured in this session
    t2_df = df[df['Time'] == 2]
    output_list.append(t2_df)
    
    # Create dataset for sessions  greater than Y05
    if max_tp >= 3:
        for i in range(3,max_tp+1):
            t_df = df[df['Time'] == max_tp]
            output_list.append(t_df)
    
    return output_list

In [ ]:
def impute_df(df):
    """
    INPUT:
    df : (dataframe)
    OUTPUT: dataframe
    DESCRIPTION: Imputes the categorical and numerical features of the given dataframe with the 'Time + Type' imputation method
    """
    # Split the dataframe by time point:
    tp_split_ls = split_df_by_time(df)

    # initialise the output list for the imputed unique sessions datasets
    imp_split_ls = []

    for df in tp_split_ls:
        # Create the lists of categorical and continuous features & get the label column.
        label_col = ['MStype']
        cat_cols = ['Sex', 'HC_CI_CP']
        num_cols = [col for col in df.columns if col not in ['Time', label_col[0], 'PRESGENE_ID', 'Age_group'] + cat_cols]

        # Save ID columns
        PRESGENE_IDs = df[['PRESGENE_ID']]
        noID_df = df.drop(columns = 'PRESGENE_ID')
        
        # Ensure numerical columns have numeric type
        for col in num_cols:
            noID_df[col] = pd.to_numeric(noID_df[col], errors='coerce')

        imputed_df = noID_df.copy()

        # Impute NaN for MStype columns.
        imputed_df['MStype'] = imputed_df['MStype'].fillna(noID_df['MStype'].mode()[0])

        # Impute NaN for categorical columns.
        for col in cat_cols:
            imputed_df[col] = imputed_df.groupby(label_col, observed=True)[col].transform(lambda x: x.fillna(x.mode()[0]))
    
        # Impute NaN for continuous columns.
        for col in num_cols:
            imputed_df[col] = imputed_df.groupby(label_col, group_keys = True)[col].transform(lambda x: x.fillna(x.mean()))

        # Ensure correct class for MS based on MStype value after imputation
        imputed_df['MS'] = imputed_df['MStype'].apply(lambda x: 2 if x == 0 else 1)
        
        # Apply rounding and ensure correct values in EDSS
        imputed_df['EDSS'] = imputed_df['EDSS'].apply(lambda x: 0 if x < 0.5 else (1 if x < 1.5 else round(x * 2) / 2 if x != 999 else 999))
        imputed_df[['Time', 'Sex', label_col[0], 'HC_CI_CP']] = imputed_df[['Time', 'Sex', label_col[0], 'HC_CI_CP']].round().astype(int)
        working_df = pd.concat([PRESGENE_IDs, imputed_df], axis = 1)

        imp_split_ls.append(working_df)
    
    return imp_split_ls

## Functions for fixing class imbalance with SMOTE

In [ ]:
def merge_time_by_rows(ls_df):
    """
    INPUT: 
    ls_df : (list of dataframes) list of dataframes with unique sessions
    OUTPUT: list of dataframes
    DESCRIPTION: Merges the datasets such that each participant has 1 row instead of n rows for the n sessions.
    """
    # Preserve `Sex` and `MS` columns only in the first DataFrame
    for idx in range(1, len(ls_df)):
        ls_df[idx] = ls_df[idx].drop(['Sex', 'MS'], axis=1, errors='ignore')

    # Combine the time point split datasets row-wise
    concat_df = pd.merge(ls_df[0], ls_df[1], on='PRESGENE_ID', how='left', 
                         suffixes=('_tp1', '_tp2'))

    # If `tp3` is included
    if len(ls_df) == 3:
        df3 = ls_df[2].rename(columns={col: col + '_tp3' for col in ls_df[2].columns if col != 'PRESGENE_ID'})
        concat_df = pd.merge(concat_df, df3, on='PRESGENE_ID', how='left')

    return concat_df

In [ ]:
def reformat_SMOTE_dfs(df):
    """
    INPUT: 
    df : (dataframes) dataframe with the 1 row per participant format created by the functionmerge_time_by_rows
    output: list with first a dataframe of the merged sessions with X rows per participant per X sessions, followed by the unique sessions
    datasets.
    DESCRIPTION: Splits the input dataframe into unique sessions, remove the session specific suffix from the column names, return a 
    list of the unique sessions dataset and a version with the merged sessions.
    """
    # Reset index to use it as 'PRESGENE_ID' substitute
    df_idx = df.reset_index(drop = False)

    # Get the column names for the time splits    
    tp1_cols = ['index', 'Sex', 'MS']
    tp2_cols = ['index', 'Sex', 'MS']
    tp3_cols = ['index', 'Sex', 'MS']

    
    for col in df_idx.columns:
        if 'tp3' in col and col not in ['index_tp3','Sex_tp3', 'MS_tp3']:
            tp3_cols.append(col)
        elif 'tp2' in col and col not in ['index_tp2','Sex_tp2', 'MS_tp2']:
            tp2_cols.append(col)
        elif 'tp1' in col:
            tp1_cols.append(col)

    # Get the n timepoints and make n new subset dfs
    df_tp1 = df_idx[tp1_cols]
    df_tp2 = df_idx[tp2_cols]
    df_tp3 = None
    
    if len(tp3_cols) > 3:
        df_tp3 = df_idx[tp3_cols]

    # Rename the columns
    df_tp1.columns = df_tp1.columns.str.replace('_tp1', '', regex=False)
    df_tp2.columns = df_tp2.columns.str.replace('_tp2', '', regex=False)
    comb_ls = [df_tp1, df_tp2]
    if df_tp3 is not None:
        df_tp3.columns = df_tp3.columns.str.replace('_tp3', '', regex=False)
        comb_ls.append(df_tp3)

    # Make the combined tp dataset
    combined_df = pd.concat(comb_ls, ignore_index=True)

    return [combined_df] + comb_ls

In [ ]:
def class_rebalancing_SMOTE(ls_df):
    """
    INPUT:
    ls_ls_df : (nested list of dataframes) nested list containing the imputed datasets, the first list for the MS unique sessions datasets 
    and a second for ALL unique sessions datasets.
    OUTPUT: lists of dataframes containing the combined sessions and the unique sessions datasets
    DESCRIPTION: Uses SMOTE to fix class imbalance in the (HC +) MS phenotypes in the datasets
    respectively. 
    """
    # Combine the unique time points row-wise
    rowco_df = merge_time_by_rows(ls_df)
    smote_ft = 'MStype_tp1'
            
    ### SMOTE
    # Define X & y
    X = rowco_df.drop([smote_ft, 'PRESGENE_ID'], axis=1)
    y = rowco_df[smote_ft]

    # Find frequency of the majority class in MS phenotypes
    resample_size = y.value_counts().max() * 2

    # Create a sampling strategy for classes with < 2 samples  
    small_classes = y.value_counts()[y.value_counts() < 2].index
    sampling_strategy = {label: resample_size for label in y.unique() if label not in small_classes}
    
    # Dynamic number of neighbors (3 or smallest class, at least 1)
    min_samples = y.value_counts().min()
    min_neighbors = max(1, min(3, min_samples - 1))

    smote_model = SMOTE(sampling_strategy = sampling_strategy, k_neighbors = min_neighbors, random_state=42) 
    X_SMOTE, y_SMOTE = smote_model.fit_resample(X, y)

    output_df = pd.DataFrame(X_SMOTE, columns = X.columns)
    output_df = pd.concat([output_df, pd.Series(y_SMOTE, name=smote_ft)], axis=1)
    
    # Reformat the extrapolated EDSS values
    edss_cols = [col for col in output_df.columns if col.startswith('EDSS')]
    for col in edss_cols:
        output_df[col] = output_df[col].apply(lambda x: 999 if x >= 10.5 else 
                                              (0 if x < 0.5 else 
                                               (1 if x < 1.5 else 
                                                round(x * 2) / 2 if x != 999 else 999)))

    # Ensure the MS and MStype columns remain integers
    label_cols = [col for col in output_df.columns if col.startswith('MS')]
    for col in label_cols:
        output_df[col] = output_df[col].round().astype(int)
        
    # Reformat the dataset from 1 row per subject to n row per n time points per subject.
    output_ls = reformat_SMOTE_dfs(output_df)

    return output_ls

## Functions for dimensionality reduction with UMAP

In [ ]:
def apply_unique_UMAP(train_df, test_df, perp, dist, metric):
    """
    INPUT:
    train_df : (Dataframe) Containing the training data for fitting and embedding.
    test_df : (Dataframe) Containing the test data for embedding.
    perp : (Integer) the perplexity value to use during UMAP embedding.
    dist : (Integer) the minimum distance value to use during UMAP embedding.
    metric : (String) name of the metric to use during UMAP embedding.
    OUTPUT: 2 dataframes with UMAP embedded features (first is the training df and then the testing df)
    DESCRIPTION: Removes columns not belonging to clinical, demographic, sMRI or dMRI before embedding. fits the UMAP embeddings to the
    train dataset and embeds the test set with the fitted UMAP. converts the array to data frames.
    is also saved with the given df name as a .csv file. 
    """
    # Get name, perplexity, learning rate and label column name for the df
    label_col = [col for col in train_df.columns if col.startswith('MS')]
    index_col = train_df.columns[0]

    # Remove target variables and normalise the train df
    norm_train = train_df.drop(columns = ['EDSS', 'BL_Avg_cognition', 'Time', 'HC_CI_CP', index_col] + label_col, axis = 1)
    norm_train = StandardScaler().fit_transform(norm_train) 
    # Remove target variables and normalise the test df
    norm_test = test_df.drop(columns = ['EDSS', 'BL_Avg_cognition', 'Time', 'HC_CI_CP', index_col] + label_col, axis = 1)
    norm_test = StandardScaler().fit_transform(norm_test) 

    # Run the UMAP model
    umap_model = UMAP(n_components = 2, n_neighbors = perp, min_dist = dist, 
                      metric = metric, random_state = 42)
    train_array = umap_model.fit_transform(norm_train)
    test_array = umap_model.transform(norm_test)

    # Create the output dfs
    train_output = pd.DataFrame(train_array)
    train_idx_df = train_df.loc[:, [index_col, 'Time']]
    train_idx_df = train_idx_df.reset_index(drop=True)
    train_output = pd.concat([train_idx_df, train_output],axis = 1)

    test_output = pd.DataFrame(test_array)
    test_idx_df = test_df.loc[:, [index_col, 'Time']]
    test_idx_df = test_idx_df.reset_index(drop=True)
    test_output = pd.concat([test_idx_df, test_output],axis = 1)

    return train_output, test_output

## Functions for Feature engineering

In [ ]:
def make_dependent(transformed_df, df, task_ft):
    """
    INPUTS :
    transformed_df : (dataframe) to which he engineered feature should be added to
    df : (data frame) smote oversampled df with combined sessions from which the features will be engineered
    task_ft : (string) the name of the dependent variable.
    OUTPUT: 1 dataframe
    DESCRIPTION: Use the existing clinical columns: EDSS, BL_Avg_cognition, MS, MStype to make the new dependent variables: bi_EDSS, 
    tri_EDSS, worsening_EDSS, and RR_to_SP. The feature corresponding to the task_ft is added to the transformed_df as the target variable. 
    """
    #Isolate the wanted columns
    index_col = transformed_df.columns[0]
    dep_df = df[[index_col, 'Time']]
    
    if task_ft == 'BL_Avg_cognition':
        # Round the average cognition score to 2 decimal points (as in the original datasets)
        dep_df.loc[:, 'BL_Avg_cognition'] = df.loc[:,'BL_Avg_cognition'].round(2) 

    elif task_ft == 'bi_EDSS':
        # Make the bi_EDSS (EDSS ≤ 5.5 = 1 lower bound for worsening cutoff. EDSS > 5.5 upper bound for worsening EDSS cutoff)
        dep_df.loc[:, 'bi_EDSS'] = df.loc[:, 'EDSS'].apply(lambda x: 0 if x > 5.5 else 1)

    elif task_ft == 'tri_EDSS':
        # Make the tri_EDSS (EDSS ≤ 3.5 = 1 (mild), 3.5 < EDSS ≤ 6.0 = 2 (moderate), EDSS ≥ 6.5 = 3 (severe)) feature, 4 = HC
        dep_df.loc[:, 'tri_EDSS'] = df.loc[:, 'EDSS'].apply(lambda x: 1 if x <= 3.5 else (2 if x > 3.5 and x <= 6.0 else (3 if x > 6.0 and x <= 10.0 else 4)))

    elif task_ft == 'worsening_EDSS':
        # Make the 'EDSS_worsening' feature (if : change ≥ 1.0 if baseline EDSS ≤ 5.5 or change ≥ 0.5 if baseline EDSS > 5.5)
        max_tp = df['Time'].unique().tolist()[-1]
        dep_df.loc[:, 'worsening_EDSS'] = df.groupby(by = index_col)['EDSS'].transform(lambda x: (
                                                                                        1 if (
                                                                                            (x[df['Time'] == max_tp].values[0] - x[df['Time'] == 1].values[0] >= 0.5 and x[dep_df['Time'] == 1].values[0] > 5.5) or 
                                                                                            (x[df['Time'] == max_tp].values[0] - x[df['Time'] == 1].values[0] >= 1.0 and x[dep_df['Time'] == 1].values[0] <= 5.5)
                                                                                        ) 
                                                                                        else 0))


    elif task_ft == 'RR_to_SP':
        max_tp = df['Time'].max()
        
        # Calculate RR_to_SP for each group
        rr_to_sp = (
            df.groupby(index_col)[['Time', 'MStype']]
            .apply(
                lambda group: int(
                    not group[group['Time'] == 1].empty and 
                    not group[group['Time'] == max_tp].empty and
                    group.loc[group['Time'] == 1, 'MStype'].values[0] == 3 and
                    group.loc[group['Time'] == max_tp, 'MStype'].values[0] == 2
                )
            )
        )
        
        # Map RR_to_SP back to all rows in the original DataFrame
        dep_df.loc[:, 'RR_to_SP'] = df[index_col].map(rr_to_sp)
        
        # Fill NaN values with 0
        dep_df.loc[:,'RR_to_SP'] = dep_df['RR_to_SP'].fillna(0).astype(int)
    
    elif task_ft == 'MS':
        # Conver MS to proper encoding: 0 if HC, 1 if MS
        dep_df.loc[:, 'MS'] = df['MS']
        dep_df.loc[dep_df['MS'] == 2, 'MS'] = 0

    elif task_ft == 'EDSS':
        # Reformat EDSS if task_ft
        dep_df.loc[:, 'EDSS'] = df['EDSS']
        dep_df.loc[:, 'EDSS'] = dep_df['EDSS'].apply(lambda x: 0 if x < 0.5 else (1 if x < 1.5 else round(x * 2) / 2 if x != 999 else 999))

    else:
        dep_df.loc[:, task_ft] = df[task_ft]

    # Prep the transformed df for merging with the dependent feature df
    dep_var_cols = [col for col in dep_df.columns if col not in [index_col, 'Time']]
    working_df = transformed_df[[col for col in transformed_df.columns if col not in dep_var_cols]]

    # Combine the dependent feature with the transformed or raw df 
    output_df = pd.merge(dep_df, working_df, on=[index_col, 'Time'], how='inner')

    return output_df 

# Step 3: Set up the LSTM

## General MLA functions

In [ ]:
class EDSSLabelEncoder:
    # Create a class for EDSS value encoding
    def __init__(self):
        # Define the mapping from float to integer
        self.mapping = {
            0.0: 0,
            1.0: 1,
            1.5: 2,
            2.0: 3,
            2.5: 4,
            3.0: 5,
            3.5: 6,
            4.0: 7,
            4.5: 8,
            5.0: 9,
            5.5: 10,
            6.0: 11,
            6.5: 12,
            7.0: 13,
            7.5: 14,
            8.0: 15,
            8.5: 16,
            9.0: 17,
            9.5: 18,
            10.0: 19,
            999.0: 20,
        }
        # Create the inverse mapping
        self.inverse_mapping = {v: k for k, v in self.mapping.items()}

    def fit_transform(self, y):
        return [self.mapping[val] for val in y]

    def transform(self, y):
        return [self.mapping[val] for val in y]

    def inverse_transform(self, y):
        return [self.inverse_mapping[val] for val in y]

In [ ]:
def rdm_oversampling(df, dep_ft):
    """
    INPUT:
    df : (Dataframe) Containing embedded data and dependent feature of choice.
    dep_ft : feature used for oversampling (target variable)
    OUTPUT: dataframe
    DESCRIPTION: Use random sampling to rebalance the classes in a df based on the input feature of choice. 
    """
    # Isolate the target variable
    X = df.drop(dep_ft, axis = 1)
    y = df[dep_ft]

    # Initialise and fit the random over sampler (ROS)
    ROS = RandomOverSampler(random_state=42)

    # Apply ROS
    X_ROS, y_ROS = ROS.fit_resample(X,y)

    # Format the output df to match the column order of the input dataframe
    col_idx = df.columns.get_loc(dep_ft)
    output_df = pd.concat([X_ROS.iloc[:, :col_idx], y_ROS, X_ROS.iloc[:, col_idx:]], axis=1)

    return output_df

In [ ]:
def evaluate_score(old_score, eval_scores_dict, old_best, new_best):
    """
    INPUTS:
    old_score: (float) value corresponding to previous eval score value
    eval_scores_dict: (dictionary) containing scores obtained in current MLA evaluation
    old_best: (dictionary or MLA model) contains the hps of the previous best model or the previous best model
    new_best: (dictionary or MLA model) contains the hps of the newest model or the previous best model
    OUTPUTS: evaluation score (float) and dictionary of hyperparameters
    DESCRIPTION: General function for comparing the output scores of MLAs. 
    """    
    # For Classification tasks
    if 'f1score' in eval_scores_dict:
        new_f1 = eval_scores_dict['f1score']
        if new_f1 > old_score: 
            score = new_f1
            best = new_best
            print(f'~~ The old F1 score was: {old_score} & the new F1 score is: {score}')
        
        else:
            score = old_score
            best = old_best
            print('The f1score was not greater, the evaluation score remains unchanged')

    # For Regression tasks
    elif 'mse' in eval_scores_dict:
        new_mse = eval_scores_dict['mse']
        if new_mse < old_score:
            score = new_mse
            best = new_best
            print(f'~~ The old MSE score was: {old_score}, the new MSE score is: {score}')

        else:
            score = old_score
            best = old_best
            print('The mse was not smaller, the evaluation score remains unchanged')

    return score, best

In [ ]:
def AUCROC(model_fitted, X_test, y_test, task_type, model_type):
    """
    INPUT:
    model_fitted : (fitted model) A model fitted to the training data
    X_test : (DataFrame) data of test subjects
    y_test : (DataFrame) column of test DataFrame with the test labels 
    task_type : (str) nature of the (classifaication) task, 'binary' or 'multiclass'
    model_type : (str) MLA abbreviated name ('SVM', 'LogR', 'RF', or 'XGB)'
    OUTPUT: AUC score for the ROC and its corresponding plot
    DESCRIPTION: Obtain and plot the ROC, and compute its AUC score for binary or multiclass tasks.
    """
    # Initialise the outputs
    roc_auc, pred_scores, output_plot, classes = None, None, None, None 

    if len(np.unique(y_test)) > 1:  #Ensures that AUC ROC is only calculated when more than 1 class is present

        # Get the prediction scores depending on the MLA
        if hasattr(model_fitted, "decision_function") or model_type == 'SVM': # For SVM
            pred_scores = model_fitted.decision_function(X_test)

            if task_type != 'binary':
                classes = list(model_fitted.classes_)
        
        elif model_type == 'LSTM':
            pred_scores = model_fitted.model.predict(X_test)
            
            if task_type == 'binary':
                pred_scores = pred_scores[:, 1]
                y_test = y_test[:, 1]

            else:
                y_test = np.argmax(y_test, axis=1)
                classes = list(range(pred_scores.shape[1]))
                print(classes)
                
        else:
            if task_type == 'binary':
                pred_scores = model_fitted.predict_proba(X_test)[:, 1]
            else:
                pred_scores = model_fitted.predict_proba(X_test)
                classes = list(model_fitted.classes_)
        
        if task_type == 'binary': # Binary ROC
            # Predict probabilities & find TPR, FPR at varying thresholds
            fpr, tpr, threshold = roc_curve(y_test, pred_scores, pos_label=1)
    
            # auc score
            roc_auc = roc_auc_score(y_test, pred_scores)
            
            # plot the ROC curve
            fig, ax = plt.subplots()
            plt.plot(fpr, tpr, linestyle = '-',color = '#1E8449', label=f'{model_type} (AUC = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], linestyle = '--', color = '#17202A', label = 'Random (AUC = 0.50)')
            plt.title(f'Binary ROC Curve for {model_type}')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive rate')
            plt.legend(loc='lower right', prop = {'size':10})
            
            output_plot = fig
    
        else: # Multiclass ROC 
            fpr, tpr, roc_auc = dict(), dict(), dict()
            fig, ax = plt.subplots()
            common_classes = [cls for cls in classes if cls in np.unique(y_test)]

            # Calculate ROC curve and AUC for each class
            if model_type == 'LSTM':
                for cls in common_classes:
                    fpr[cls], tpr[cls], _ = roc_curve(y_test == cls, pred_scores[:, cls])
                    roc_auc[cls] = roc_auc_score(y_test == cls, pred_scores[:, cls])

            else:
                for cls in common_classes:
                    fpr[cls], tpr[cls], _ = roc_curve(y_test == cls, pred_scores[:, classes.index(cls)])
                    roc_auc[cls] = roc_auc_score(y_test == cls, pred_scores[:, classes.index(cls)])
        
            # Plot ROC curves
            colors = [ 'pink', 'orange', 'purple', '#C0392B', '#2980B9', '#3498DB', '#1ABC9C', '#16A085', '#27AE60', '#F1C40F', '#E67E22', 
                     '#D35400', '#F5B7B1', '#D7BDE2', '#AED6F1', '#A3E4D7', '#A9DFBF', '#F9E79F', '#FAD7A0', '#EDBB99', '#641E16',
                     '#4A235A', '#154360', '#117864', '#196F3D']
            for cls, color in zip(common_classes, colors):
                plt.plot(fpr[cls], tpr[cls], color=color, lw=2, label=f'Class {cls} (AUC = {roc_auc[cls]:.2f})')
                
            plt.plot([0, 1], [0, 1], color='#17202A', lw=2, linestyle='--', label='Random (AUC = 0.50)')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.title(f'Multiclass ROC Curve for {model_type}')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.legend(loc='lower right', prop = {'size':7})
    
            output_plot = fig

        return roc_auc, output_plot

    else:
        print('Only 1 class was present in the test set, no aucroc will be returned.')
        
        return {}, None

## Functions for the LSTM

In [ ]:
def prep_LSTM_data(df, task_ft):
    """
    INPUT:
    df : (dataframe)
    task_ft : (String) name of the column to use as target variable.
    OUTPUT: dataset
    DESCRIPTION: Removes irrelevant columns and restructures the dataset to 1 row per subject (structure needed for LSTM input).
    """
    ## Split the dataset per time point
    index_col = df.columns[0]
    unique_tps = df['Time'].unique().tolist()
    max_tp = max(unique_tps)
    dep_var_ls = ['MS', 'MStype', 'EDSS', 'BL_Avg_cognition', 'HC_CI_CP', 'bi_EDSS',
              'tri_EDSS', 'worsening_EDSS', 'RR_to_SP']
    dep_var_ls.remove(task_ft)

    filtered_df = df[[col for col in df.columns if col not in dep_var_ls]]

    #Make tp 1 base datset
    working_df = filtered_df[filtered_df['Time'] == 1]
    working_df = working_df.drop([task_ft], axis = 1)
    
    ## Combine the time points such that each subject only has 1 row
    for tp in unique_tps:
        if tp == 2:
            ### Combine the time points for the datasets
            working_df = working_df.merge(filtered_df[filtered_df['Time'] == tp], on=index_col, how='outer',
                                     suffixes=('_tp1', '_tp2'))

            if max_tp > 2:
                    working_df = working_df.drop([task_ft], axis = 1)

        elif tp > 2:  
            ### Add the appropriate suffix based on the corresponding time point for time points greater than 2
            add_working_df = filtered_df[filtered_df['Time'] == tp].rename(columns={col: str(col) + '_tp' + str(tp) for col in filtered_df.columns if col not in [index_col, task_ft]})

            ### Combine the time points for the datasets
            working_df = working_df.merge(add_working_df, on=index_col, how='outer')

            if max_tp > tp:
                working_df = working_df.drop([task_ft], axis = 1)
    
    ## Remove the time columns as they will not be needed further
    working_df = working_df.drop([col for col in working_df.columns if col.startswith('Time') or col == index_col], axis = 1)

    return working_df

In [ ]:
def tune_lstm(train_df, test_df, task_ft, transformation, num_tps, outer_run, inner_run):
    """
    INPUT:
    train_df : (dataframe) dataframe of embedded training values
    test_df : (dataframe) dataframe of embedded testing values
    task_ft : (String) name of the column to use as label column.
    transformation: (string) The mthod of transformation to apply to the dataset (None/UMAP)
    num_tps: (integer) number of the highest session to include (0 = Y00, 1 = Y05, 3 = Y10)
    outer_run: (integer) current outer run
    inner_run: (integer) current inner run
    OUTPUT:  float of the evaluation metrics and the corresponding hp
    DESCRIPTION: Tunes the LSTM for the inner runs of the nCV
    """
    index_col = train_df.columns[0]
    num_tps = len(train_df['Time'].unique().tolist())
    output_score = {}
    dep_cols = ['PRESGENE_ID', 'Time', 'MS', 'MStype', 'BL_Avg_cognition', 'HC_CI_CP','bi_EDSS', 'tri_EDSS', 'worsening_EDSS', 'RR_to_SP', 'EDSS']

    if len(train_df['Time'].unique()) > 1: # Multiple tps need 1 row per subject rearrangement
        train_df = prep_LSTM_data(train_df, task_ft)
        test_df = prep_LSTM_data(test_df, task_ft)
        print('Datasets now have 1 row per subject')
    
    # Determine task type (Regression or Classification)
    if task_ft in ['BL_Avg_cognition']:
        print('The task for this run is: Regression')
        # Bootstrapping
        n_samples = train_df.shape[0] * 2
        bstrp_df = resample(train_df, replace=True, n_samples = n_samples, random_state=42)
        #print(f" the new number of rows of the df after bootstrapping is {bstrp_df.shape[0]} compared to the initial {train_df.shape[0]} rows")

        # Define variables
        temp_X_train = bstrp_df[[col for col in bstrp_df.columns if col not in dep_cols]].values
        X_train = temp_X_train.reshape((temp_X_train.shape[0], num_tps, int(temp_X_train.shape[1]/num_tps)))
        y_train = bstrp_df[[task_ft]].values
        
        temp_X_test = test_df[[col for col in bstrp_df.columns if col not in dep_cols]].values
        X_test = temp_X_test.reshape((temp_X_test.shape[0], num_tps, int(temp_X_test.shape[1]/num_tps)))
        y_test = test_df[[task_ft]].values

        # define LSTM creator for the random search
        def lstm_regressor_inner(hp):
            model = Sequential()
            model.add(LSTM(hp.Int('input_unit', min_value=10, max_value=400, step=20), return_sequences=True, 
                           input_shape=(X_train.shape[1], X_train.shape[2])))
        
            # Varying number of layers
            for i in range(hp.Int('n_layers', 1, 6)):
                model.add(LSTM(hp.Int(f'lstm_{i}_units', min_value=10, max_value=400, step=20), return_sequences=True))
                
                if random.choice([True, False]): # Randomly include dropout statement or not
                    model.add(Dropout(hp.Float(f'Dropout_rate_for_unit_{i}', min_value=0, max_value=0.5, step=0.1)))
        
            model.add(LSTM(hp.Int('layer_final_neurons', min_value=10, max_value=400, step=20)))
            
            if random.choice([True, False]):
                model.add(Dropout(hp.Float('Dropout_rate_final', min_value=0, max_value=0.5, step=0.1)))
                
            model.add(Dense(1, activation=hp.Choice('dense_activation', values=['relu', 'sigmoid', 'linear'], default='linear'))) 
            model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
        
            return model
    
        # Random search
        regression_tuner = RandomSearch(
            lstm_regressor_inner, 
            objective='mse', 
            max_trials=10, 
            executions_per_trial=1, 
            directory=f'updated_data/nested_CV_final/LSTM/{transformation}/models', 
            project_name=f'model_{transformation}{num_tps}{outer_run}{inner_run}{task_ft}')

        regression_tuner.search(x=X_train, y=y_train, epochs=10, validation_data=(X_test, y_test))

        # Get best MSE & hps
        best_model = regression_tuner.get_best_models(num_models=1)[0]
        _, output_score['mse'] = best_model.evaluate(X_test, y_test, verbose=0)
        best_hps = regression_tuner.get_best_hyperparameters(num_trials=1)[0]
    
    else: # Classification Task
        print(f'The task for this run is: Classification, the task feature is: {task_ft}')
        rs_df_train = train_df.copy() 
        rs_df_train.columns = rs_df_train.columns.astype(str)
        rs_df_test = test_df.copy()
        rs_df_test.columns = rs_df_test.columns.astype(str)
        label_encoder = None
        
        # Class Rebalancing with random over sampling
        if task_ft == 'EDSS':
            label_encoder = EDSSLabelEncoder()
            rs_df_train['EDSS'] = label_encoder.fit_transform(rs_df_train['EDSS'])
            rs_df_test['EDSS'] = label_encoder.transform(rs_df_test['EDSS'])

            rs_df_train = rdm_oversampling(rs_df_train, task_ft)

        elif len(set(Counter(train_df[task_ft].values))) != 1:
            rs_df_train = rdm_oversampling(rs_df_train, task_ft)
            print(f'the rbalanced classes are: {rs_df_train[task_ft].value_counts()}')

        # Define variables
        temp_X_train = rs_df_train[[col for col in rs_df_train.columns if col not in dep_cols]].values
        X_train = temp_X_train.reshape((temp_X_train.shape[0], num_tps, int(temp_X_train.shape[1]/num_tps)))

        temp_X_test = rs_df_test[[col for col in rs_df_test.columns if col not in dep_cols]].values
        X_test = temp_X_test.reshape((temp_X_test.shape[0], num_tps, int(temp_X_test.shape[1]/num_tps)))

        ## Get the one hot encoded y_train & y_test
        ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
        y_train = ohe.fit_transform(rs_df_train[[task_ft]].values)
        y_test = ohe.transform(rs_df_test[[task_ft]].values)
        
        dense_output = y_train.shape[1]

        # define LSTM creator for the random search
        def lstm_classifier_inner(hp):
            model = Sequential()
        
            model.add(LSTM(hp.Int('input_unit',min_value=10,max_value=400,step=20),
                           return_sequences=True, 
                           input_shape=(X_train.shape[1],X_train.shape[2])))
           
            for i in range(hp.Int('n_layers', 1, 6)):
                model.add(LSTM(hp.Int(f'lstm_{i+1}_units',min_value=10,max_value=400,step=20),return_sequences=True))
        
                if random.choice([True, False]): # Randomly include dropout statement or not
                    model.add(Dropout(hp.Float(f'Dropout_rate_for_unit_{i}', min_value=0, max_value=0.5, step=0.1)))
        
            model.add(LSTM(hp.Int('layer_final_neurons', min_value=10, max_value=400, step=20),return_sequences=False))
            
            if random.choice([True, False]):
                model.add(Dropout(hp.Float('Dropout_rate_final', min_value=0, max_value=0.5, step=0.1)))
            
            model.add(Dense(dense_output, activation='softmax'))
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
            
            return model
    
        # Random search
        classifier_tuner = RandomSearch(
            lstm_classifier_inner, 
            objective= 'val_accuracy',
            max_trials=10, 
            executions_per_trial=1, 
            directory=f'updated_data/nested_CV_final/LSTM/{transformation}/models', 
            project_name=f'model_{transformation}{num_tps}{outer_run}{inner_run}{task_ft}')

        classifier_tuner.search(x=X_train, y=y_train, epochs=10, validation_data=(X_test, y_test))

        # Get best MSE & hps
        best_model = classifier_tuner.get_best_models(num_models=1)[0]
        output_score['mse'], acc = best_model.evaluate(X_test, y_test, verbose=0)
        best_hps = classifier_tuner.get_best_hyperparameters(num_trials=1)[0]

    print('val loss:', output_score['mse'])

    return output_score, best_hps

In [ ]:
def lstm_regressor_outer(best_hps, X_train):
    """
    INPUT:
    best_hps : (dictionary) contains the hps to build the LSTM with
    X_train:  (dataframe) train dataset without target variable
    OUTPUT: LSTM Regressor
    DESCRIPTION: Create the LSTM regressor based on the given hps and the shape of the input train dataset for the outer runs of the nCV
    """
    model = Sequential()
    model.add(LSTM(best_hps['input_unit'], return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    
    # Adding layers based on the hyperparameters
    for i in range(best_hps['n_layers']):
        model.add(LSTM(best_hps[f'lstm_{i}_units'], return_sequences=True))
        
        if f'Dropout_rate_for_unit_{i}' in best_hps:
            model.add(Dropout(best_hps[f'Dropout_rate_for_unit_{i}']))
    
    model.add(LSTM(best_hps['layer_final_neurons']))
    
    if 'Dropout_rate_final' in best_hps:
        model.add(Dropout(best_hps['Dropout_rate_final']))
    
    model.add(Dense(1, activation=best_hps['dense_activation']))
    
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    
    return model

In [ ]:
def lstm_classifier_outer(best_hps, X_train, dense_output):
    """
    INPUT:
    best_hps : (dictionary) contains the hps to build the LSTM with
    X_train:  (dataframe) train dataset without target variable
    dense_output: (integer) number of classes in y_train (the target variable)
    OUTPUT: LSTM Classifier
    DESCRIPTION: Create the LSTM classifier based on the given hps and the shape of the input train dataset for the outer runs of the nCV
    """
    model = Sequential()
    model.add(LSTM(best_hps['input_unit'], return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
   
    for i in range(best_hps['n_layers']):
        model.add(LSTM(best_hps[f'lstm_{i+1}_units'], return_sequences=True))
        
        if f'Dropout_rate_for_unit_{i}' in best_hps:
            model.add(Dropout(best_hps[f'Dropout_rate_for_unit_{i}']))

    model.add(LSTM(best_hps['layer_final_neurons']))
    
    if 'Dropout_rate_final' in best_hps:
        model.add(Dropout(best_hps['Dropout_rate_final']))
    
    model.add(Dense(dense_output, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
def run_lstm(train_df, test_df, task_ft, model_params, transformation, num_tps, outer_run):
    """
    INPUT:
    train_df : (dataframe) dataframe of embedded training values
    test_df : (dataframe) dataframe of embedded testing values
    task_ft : (String) name of the column to use as label column.
    transformation: (string) The mthod of transformation to apply to the dataset (None/UMAP)
    num_tps: (integer) number of the highest session to include (0 = Y00, 1 = Y05, 3 = Y10)
    outer_run: (integer) current outer run
    inner_run: (integer) current inner run
    OUTPUT:  float of the evaluation metrics and the corresponding hps
    DESCRIPTION: Builds and evaluates the LSTM for the outer runs of the nCV
    """
    index_col = train_df.columns[0]
    max_tp = len(train_df['Time'].unique().tolist())
    output_dict = {}
    auc_plot = None
    dep_cols = ['PRESGENE_ID', 'Time', 'MS', 'MStype', 'BL_Avg_cognition', 'HC_CI_CP','bi_EDSS', 'tri_EDSS', 'worsening_EDSS', 'RR_to_SP', 'EDSS']
    
    if len(train_df['Time'].unique()) > 1: # Multiple tps need 1 row per subject rearrangement
        train_df = prep_LSTM_data(train_df, task_ft)
        test_df = prep_LSTM_data(test_df, task_ft)
        print('Datasets now have 1 row per subject')
    
    # Determine task type (Regression or Classification)
    if task_ft in ['BL_Avg_cognition']:
        print('The task for this run is: Regression')
        # Bootstrapping
        n_samples = train_df.shape[0] * 2
        bstrp_df = resample(train_df, replace=True, n_samples = n_samples, random_state=42)
        #print(f" the new number of rows of the df after bootstrapping is {bstrp_df.shape[0]} compared to the initial {train_df.shape[0]} rows")

        # Define variables
        temp_X_train = bstrp_df[[col for col in bstrp_df.columns if col not in dep_cols]].values
        X_train = temp_X_train.reshape((temp_X_train.shape[0], max_tp, int(temp_X_train.shape[1]/max_tp)))
        y_train = bstrp_df[[task_ft]].values
        
        temp_X_test = test_df[[col for col in bstrp_df.columns if col not in dep_cols]].values
        X_test = temp_X_test.reshape((temp_X_test.shape[0], max_tp, int(temp_X_test.shape[1]/max_tp)))
        y_test = test_df[[task_ft]].values

        # Build the LSTM
        lstm = lstm_regressor_outer(model_params, X_train)
        LSTM_fitted = lstm.fit(X_train, y_train, epochs= 10)
    
        # Model evaluation
        y_pred = LSTM_fitted.model.predict(X_test)
        output_dict = {
            "mse": mean_squared_error(y_test, y_pred),
            "r2": r2_score(y_test, y_pred),
            "explained_variance": explained_variance_score(y_test, y_pred)}
        
    else: # Classification Task
        print('The task for this run is: Classification')
        rs_df_train = train_df.copy() 
        rs_df_train.columns = rs_df_train.columns.astype(str)
        rs_df_test = test_df.copy()
        rs_df_test.columns = rs_df_test.columns.astype(str)
        label_encoder = None
        
        # Class Rebalancing with random oversampling
        if task_ft == 'EDSS':
            label_encoder = EDSSLabelEncoder()
            rs_df_train['EDSS'] = label_encoder.fit_transform(rs_df_train['EDSS'])
            rs_df_test['EDSS'] = label_encoder.transform(rs_df_test['EDSS'])

            rs_df_train = rdm_oversampling(rs_df_train, task_ft)

        elif len(set(Counter(train_df[task_ft].values))) != 1:
            rs_df_train = rdm_oversampling(rs_df_train, task_ft)

        # Define variables
        temp_X_train = rs_df_train[[col for col in rs_df_train.columns if col not in dep_cols]].values
        X_train = temp_X_train.reshape((temp_X_train.shape[0], max_tp, int(temp_X_train.shape[1]/max_tp)))

        temp_X_test = rs_df_test[[col for col in rs_df_train.columns if col not in dep_cols]].values
        X_test = temp_X_test.reshape((temp_X_test.shape[0], max_tp, int(temp_X_test.shape[1]/max_tp)))

        ## Get the one hot encoded y_train & y_test
        ohe = OneHotEncoder(handle_unknown='ignore', sparse = False)
        y_train = ohe.fit_transform(rs_df_train[[task_ft]].values)
        y_test = rs_df_test[[task_ft]].values
        y_test_ohe = ohe.transform(y_test)

        dense_output = y_train.shape[1]

        # Build the LSTM
        lstm = lstm_classifier_outer(model_params, X_train, dense_output)
        LSTM_fitted = lstm.fit(X_train, y_train, epochs= 10)

        # Make predictions & evaluate       
        y_pred_probs = LSTM_fitted.model.predict(X_test)
        y_pred_max = np.argmax(y_pred_probs, axis=1)
        y_pred = ohe.inverse_transform(np.eye(ohe.categories_[0].shape[0])[y_pred_max])

        task_type = 'binary' if train_df[task_ft].nunique() <= 2 else 'weighted'
        auc_score, auc_plot = AUCROC(LSTM_fitted, X_test, y_test_ohe, task_type, 'LSTM')

        print('auc plot is', isinstance(auc_plot, plt.Figure))
        output_dict = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred, average=task_type),
            "recall": recall_score(y_test, y_pred, average=task_type),
            "f1score": f1_score(y_test, y_pred, average=task_type),
            "auc": auc_score}

        print('output_dict', output_dict, 'LSTM_fitted', LSTM_fitted, 'auc_plot', auc_plot)
    
    return output_dict, LSTM_fitted, auc_plot

# Step 4: Combine the Functions in the nCV Pipeline

## General Functions for nCV

In [ ]:
def get_unique_subs(basedf, task_ft):
    """
    INPUT:
    basedf : (dataframe)
    task_ft:  (string) name of the target variable for the current task
    OUTPUT: A dataframe with 1 column containing the unique subjects qualified for Kfold splitting for the current target variable
    with the string containing the updated task feature 
    DESCRIPTION: Obtains the participants suitable for stratifiedKfold for the current target variable and the name of the feature
    to stratify the subjects based on the original task feature
    """
    unique_subs = None
    sub_split_ft = ''

    if task_ft == 'EDSS' or 'EDSS' in task_ft:
        # Update the task_ft for subject splitting
        sub_split_ft = 'EDSS'
        
        # Isolate the PRESGENE_ID and subject splitting feature column
        unique_subs = basedf[['PRESGENE_ID', sub_split_ft]].copy()
        
        # Fill missing value with mode class temporarily for subject splitting
        mode_class = unique_subs[sub_split_ft].mode()[0]
        unique_subs[sub_split_ft] = unique_subs[sub_split_ft].fillna(mode_class)
    
        # Encode the EDSS values 
        unique_subs[sub_split_ft] = EDSSLabelEncoder().fit_transform(unique_subs['EDSS'])

        # Remove classes with a frequency of less than 3:
        class_counts = unique_subs['EDSS'].value_counts()
        unique_subs = unique_subs[unique_subs['EDSS'].isin(class_counts[class_counts > 2].index)]
    
        # Keep unique subject IDs
        unique_subs = unique_subs.drop_duplicates(subset = ['PRESGENE_ID'], keep = 'first')

    elif task_ft == 'BL_Avg_cognition': # Continuous features
        # Update the task_ft for subject splitting
        sub_split_ft = 'binned_continuous'

        # Isolate the PRESGENE_ID column
        unique_subs = basedf[['PRESGENE_ID']].copy()

        # Fill in missing values with mean temporarily for subject splitting
        basedf[task_ft] = basedf[task_ft].fillna(basedf[task_ft].mean())

        # Convert continuous variable into categorical for subject splitting (binning)
        bins = pd.qcut(basedf[task_ft], q=3, labels=False)

        # Add new bins to subject splitting dataframe
        unique_subs['binned_continuous'] = bins

        # Keep unique subject IDs
        unique_subs = unique_subs.drop_duplicates(subset = ['PRESGENE_ID'], keep = 'last')

    elif task_ft == 'RR_to_SP':
        # Update the task_ft for subject splitting
        sub_split_ft = 'MStype'
        
        # Isolate the PRESGENE_ID and subject splitting/ task feature column
        unique_subs = basedf[['PRESGENE_ID', sub_split_ft]].copy()
        
        # Fill missing value with mode class temporarily for subject splitting
        mode_class = basedf[sub_split_ft].mode()[0]
        unique_subs[sub_split_ft] = unique_subs[sub_split_ft].fillna(mode_class)

        # Remove classes with a frequency of less than 3:
        class_counts = basedf[sub_split_ft].value_counts()
        unique_subs = unique_subs[unique_subs[sub_split_ft].isin(class_counts[class_counts > 2].index)]

        # Convert the task feature column to integer (format needed for stratifiedKFold)
        unique_subs[sub_split_ft] = unique_subs[sub_split_ft].astype('Int64')

        # Keep unique subject IDs
        unique_subs = unique_subs.drop_duplicates(subset = ['PRESGENE_ID'], keep = 'first')

        
    else: # other categorical features
        # Update the task_ft for subject splitting
        sub_split_ft = task_ft
        
        # Isolate the PRESGENE_ID and subject splitting/ task feature column
        unique_subs = basedf[['PRESGENE_ID', task_ft]].copy()
        
        # Fill missing value with mode class temporarily for subject splitting
        mode_class = basedf[task_ft].mode()[0]
        unique_subs[task_ft] = unique_subs[task_ft].fillna(mode_class)

        # Remove classes with a frequency of less than 3:
        class_counts = basedf[task_ft].value_counts()
        unique_subs = unique_subs[unique_subs[task_ft].isin(class_counts[class_counts > 2].index)]

        # Convert the task feature column to integer (format needed for stratifiedKFold)
        unique_subs[task_ft] = unique_subs[task_ft].astype('Int64')

        # Keep unique subject IDs
        unique_subs = unique_subs.drop_duplicates(subset = ['PRESGENE_ID'], keep = 'first')
        
    return unique_subs, sub_split_ft

In [ ]:
def merge_time_imputed(ls_df):
    """
    INPUT: 
    ls_df : (list of dataframes) list of dataframes with unique sessions values
    OUTPUT: a list with the conbined sesions dataframe and the unique sessions dataframe
    DESCRIPTION: Merges the unique sessions dataset to obtain 1 dataset with all sessions in it.

    """
    # Initialisation of output list    
    output_df = None
    max_tp = len(ls_df)

    # Identify common columns between the first two dataframes
    common_columns = ls_df[0].columns.intersection(ls_df[1].columns)
    temp_df1 = ls_df[0][common_columns]
    temp_df2 = ls_df[1][common_columns]
    
    # Check if there is a third time point
    if max_tp > 2:
        concat_extras = []
        for i in range(2, max_tp):
            # Find common columns between the existing common columns and the nth dataframe
            common_columns = common_columns.intersection(ls_df[i].columns)
            temp_df = ls_df[i][common_columns]
            concat_extras.append(temp_df)
        # Combine all subsets
        output_df = pd.concat([temp_df1, temp_df2]+concat_extras, axis=0)
    
    else:
        # Combine only the first two subsets
        output_df = pd.concat([temp_df1, temp_df2], axis=0)
        
    return [output_df] + ls_df

In [ ]:
def apply_transform_labeled(smote_inner_train_ls, smote_inner_test_ls, transformation, task_ft):
    """
    INPUT:
    smote_inner_train_ls : (list of dataframes) list containing the training datasets
    smote_inner_test_ls:  (list of dataframes) list containing the testing datasets
    transformation: (string) method of transformation None/UMAP
    task_ft: (string) name of the target variable
    OUTPUT: 2 lists of dataframes the first for the train datasets and the second for the test datasets like the input tarin and test lists.
    DESCRIPTION: Applies UMAP or no transformation to the input datasets based on the requested transformation. Applies it to both the train
    and test datasets fo the input lists. Then passes the datasets to the feature engineering function where the target variable is either
    created or reformatted if it already existed in the base dataframe (e.g. MS, MStype...)
    """
    # Initialise the output lists
    dep_dr_inner_train_ls, dep_dr_inner_test_ls = [], []

    if transformation == 'UMAP': # UMAP transformation
        for i, smote_inner_train_df in enumerate(smote_inner_train_ls): # Iterate over the train and test input lists
            perp = 20 if task_ft in ['MS', 'HC_CI_CP', 'BL_Avg_cognition'] else 5
            dist = 0 if task_ft in ['MS', 'HC_CI_CP', 'BL_Avg_cognition'] else 0.5
            train_df, test_df = apply_unique_UMAP(smote_inner_train_df, smote_inner_test_ls[i], perp, dist, 'euclidean')
            
            # Add engineered features if relevant for the train dataset
            labeled_train_df = make_dependent(train_df, smote_inner_train_ls[0], task_ft)
            dep_dr_inner_train_ls.append(labeled_train_df)
            
            # Add engineered features if relevant for the test dataset
            labeled_test_df = make_dependent(test_df, smote_inner_test_ls[0], task_ft)
            dep_dr_inner_test_ls.append(labeled_test_df)
            

    elif transformation == 'RAW':
        for i, smote_inner_train_df in enumerate(smote_inner_train_ls):
            # Add engineered features if relevant for the train dataset
            labeled_train_df = make_dependent(smote_inner_train_df, smote_inner_train_ls[0], task_ft)
            dep_dr_inner_train_ls.append(labeled_train_df)

            # Add engineered features if relevant for the test dataset
            labeled_test_df = make_dependent(smote_inner_test_ls[i], smote_inner_test_ls[0], task_ft)
            dep_dr_inner_test_ls.append(labeled_test_df)

    else :
        print('The chosen method is not an appropriate transformation technique, choose from RAW or UMAP')

    return dep_dr_inner_train_ls, dep_dr_inner_test_ls

## Main function for nCV

In [ ]:
def CV_pipeline(dMRI, sMRI_vol, sMRI_ls, num_tps, test_tp, num_outer_folds, num_inner_fold, transformation, task_ft):
    """
    INPUTS:
    dMRI:(dataframe) dataset with dMRI data
    sMRI_vol: (dataframe) dataset with demographic, clinical and structural MRI dataset.
    sMRI_ls: (nested list of dfs) Each session has its own sublist containing the left and right hemisphere sMRI thickness 
    data within the main list.
    num_tps: (integer) number of the highest session to include (0 = Y00, 1 = Y05, 3 = Y10)
    test_tp: (integer) the number of the session to use as the test set (0 = Y00, 1 = Y05, 3 = Y10, 4 = Combined sessions)
    num_outer_folds: (integer) the number of outer folds to split the data into
    num_inner_fold: (integer) the number of inner folds to split each outer fold into
    transformation: (string) The mthod of transformation to apply to the dataset (None/UMAP)
    task_ft: (string) target variable name
    OUTPUTs: dictionary of the scores and hps obtained during the outer runs, dictionary of the scores and 
    hps obtained during the inner runs and the best model.
    DESCRIPTION: Groups all of the functions needed to run the nCV pipeline for LSTM.
    """
    # Make the base data frame from the individual input data frames
    tps = 2 if num_tps == 2 else 3
    base_df = make_base_df(dMRI, sMRI_vol, sMRI_ls, tps, task_ft)

    unique_subs, sub_split_ft  = get_unique_subs(base_df, task_ft)
    print(f'~~ The unique subjects have been obtained. The subject splitting feature is {sub_split_ft}')
    
    # Initialize the nested cross-validation with stratification
    outer_cv = StratifiedKFold(n_splits=num_outer_folds, shuffle=True, random_state=42)
    inner_cv = StratifiedKFold(n_splits=num_inner_fold, shuffle=True, random_state=42)

    outer_runs_dict, inner_runs_dict = {}, {}
    outer_eval = float('inf') if task_ft in ['BL_Avg_cognition'] else float('-inf')
    best_model = None

    for outer_run, (train_outer_idx, test_outer_idx) in enumerate(outer_cv.split(unique_subs['PRESGENE_ID'], unique_subs[sub_split_ft])):
        print(f'~~ The current OUTER run is: {outer_run + 1} / {num_outer_folds} for task {task_ft}')
        # Get train and test subject IDs (outer loop)
        outer_train_subs = unique_subs.iloc[train_outer_idx]
        outer_test_subs = unique_subs.iloc[test_outer_idx]

        print('~~ The OUTER Subject split has been obtained.')
        
        inner_eval = float('inf') if task_ft in ['BL_Avg_cognition'] else float('-inf')
        inner_hps = None
  
        for inner_run, (train_inner_idx, test_inner_idx) in enumerate(inner_cv.split(outer_train_subs['PRESGENE_ID'], outer_train_subs[sub_split_ft])):
            
            print(f'~~ The current INNER run is: {inner_run + 1} / {num_inner_fold} for outer run: {outer_run + 1} / {num_outer_folds} for task {task_ft}')
            # Get train and test subject IDs (inner loop)
            inner_train_subs = outer_train_subs.iloc[train_inner_idx]
            inner_test_subs = outer_train_subs.iloc[test_inner_idx]
            
            # Get all occurrences of the selected subjects
            inner_train_mask = base_df['PRESGENE_ID'].isin(inner_train_subs['PRESGENE_ID'])
            inner_test_mask = base_df['PRESGENE_ID'].isin(inner_test_subs['PRESGENE_ID'])
    
            # Get the training and test data
            train_inner, test_inner = base_df[inner_train_mask], base_df[inner_test_mask]
        
            print('~~ The INNER Subject split has been obtained.')
            
            ### Data Preprocessing
            # Impute missing values
            imp_inner_train_ls = impute_df(train_inner)
            imp_inner_test_ls = impute_df(test_inner)
            print('~~ The INNER imputation has been done.')                
    
            # Balance out minority classes with SMOTE for categorical variables only
            if task_ft == 'BL_Avg_cognition':
                smote_inner_train_ls = merge_time_imputed(imp_inner_train_ls)
                smote_inner_test_ls = merge_time_imputed(imp_inner_test_ls)
    
            else:
                smote_inner_train_ls = class_rebalancing_SMOTE(imp_inner_train_ls)
                smote_inner_test_ls = class_rebalancing_SMOTE(imp_inner_test_ls)
            print('~~ The INNER SMOTE Class rebalancing is done.')
    
            
            # Apply dimensionality reduction if wanted and add dependent variables
            dep_dr_inner_train_ls, dep_dr_inner_test_ls = apply_transform_labeled(smote_inner_train_ls, smote_inner_test_ls, 
                                                                                  transformation, task_ft)
            print(f'the NAs after dim reduction {dep_dr_inner_train_ls[0].isna().any().sum()}, test {dep_dr_inner_test_ls[0].isna().any().sum()}')
    
            print('~~ INNER dimensionality reduction & dependent features have been applied/added.')
    
            working_train_inner, working_test_inner = None, None
    
            #### LSTM & Evaluation
            working_train_inner = dep_dr_inner_train_ls[0] if num_tps == 3 or num_tps == 2 else dep_dr_inner_train_ls[1]
            working_test_inner = dep_dr_inner_test_ls[0] if num_tps == 3 or num_tps == 2 else dep_dr_inner_test_ls[test_tp]
            new_score, new_hps = tune_lstm(working_train_inner, working_test_inner, task_ft, transformation, num_tps, outer_run, inner_run)
            inner_eval, inner_hps = evaluate_score(inner_eval, new_score, inner_hps, new_hps)
            
            inner_runs_dict[f'{outer_run + 1}{inner_run + 1}'] = {'eval_scores': inner_eval, 'best_hps': inner_hps}
                    
            print(f'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ INNER RUN {inner_run + 1} of outer run {outer_run + 1} is completed , best inner eval score: {inner_eval} , with best hps: {inner_hps} for task {task_ft} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
            ### Outer loop: Use the best inner hyperparameters to train and evaluate the model
            
        # Get the outer loop datasets
        outer_train_mask = base_df['PRESGENE_ID'].isin(outer_train_subs['PRESGENE_ID'])
        outer_test_mask = base_df['PRESGENE_ID'].isin(outer_test_subs['PRESGENE_ID'])
        train_outer, test_outer = base_df[outer_train_mask], base_df[outer_test_mask]
        
        print('~~ The OUTER dataset split has been obtained.')
         
        ### Data Preprocessing
        # Impute missing values
        imp_outer_train_ls = impute_df(train_outer)
        imp_outer_test_ls = impute_df(test_outer)
        print('~~ The OUTER imputation has been done.')
        
        # Balance out minority classes with SMOTE
        if task_ft == 'BL_Avg_cognition':
            smote_outer_train_ls = merge_time_imputed(imp_outer_train_ls)
            smote_outer_test_ls = merge_time_imputed(imp_outer_test_ls)
        else:
            smote_outer_train_ls = class_rebalancing_SMOTE(imp_outer_train_ls)
            smote_outer_test_ls = class_rebalancing_SMOTE(imp_outer_test_ls)
        print('~~ The OUTER SMOTE Class rebalancing is done.')
        
        # Apply dimensionality reduction if wanted and add dependent variables
        dep_dr_outer_train_ls, dep_dr_outer_test_ls = apply_transform_labeled(smote_outer_train_ls, smote_outer_test_ls,
                                                                              transformation, task_ft)
        
        print('~~ OUTER Missing values have been imputed, Class rebalancing has been performed and dimensionality reduction & dependent features have been applied/added.')
        
        #### LSTM & Evaluation
        working_train_outer, working_test_outer, outer_eval_scores_dict = None, None, None
        
        #### LSTM & Evaluation
        working_train_outer = dep_dr_outer_train_ls[0] if num_tps == 3 or num_tps == 2 else dep_dr_outer_train_ls[1]
        working_test_outer = dep_dr_outer_test_ls[0] if num_tps == 3 or num_tps == 2 else dep_dr_outer_test_ls[test_tp]
        outer_eval_scores_dict, LSTM_fitted, auc_plot = run_lstm(working_train_outer, working_test_outer, task_ft, inner_hps, transformation, num_tps, outer_run)
        outer_eval, best_model = evaluate_score(outer_eval, outer_eval_scores_dict, best_model, LSTM_fitted)

        # Store the evaluation scores and hyperparameter
        outer_runs_dict[outer_run + 1] = {'eval_scores': outer_eval_scores_dict, 'best_hps': inner_hps}
        
        # Save the AUCROC plot (only for categorical variables) & close it once saved
        if isinstance(auc_plot, plt.Figure):
            auc_plot.savefig(f'updated_data/nested_CV_final/LSTM/{transformation}/AUCROC_nestCV_run{outer_run + 1}_{transformation}_{task_ft}{num_tps}.png')
            plt.close(auc_plot)
            
        print(f'~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ OUTER RUN {outer_run + 1} is completed, best outer eval score: {outer_eval} & best model {best_model} for task {task_ft} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    print('Exited all nCV loops')
    return outer_runs_dict, inner_runs_dict, best_model

# Step 5: Run the nCV

In [ ]:
# Define the parameters for the nCV run
transformation = '' # Options: RAW/UMAP
dep_vars_ls = ['EDSS', 'MS', 'MStype', 'BL_Avg_cognition', 'HC_CI_CP','bi_EDSS', 'tri_EDSS', 'worsening_EDSS', 'RR_to_SP']
n_ses = 3 # Number of sessions to include in the training data (1, 2, or 3)
n_test = 3 # Session to use as test set (1, 2, or 3)
n_outer = 3 # Number of outer nCV runs (> 1)
n_inner = 3 #Number of inner nCV runs (> 1)

In [ ]:
# Run the nCV with the given parameters
filepath = f'updated_data/nested_CV_final/LSTM/{transformation}'
master_path = f'{filepath}/All3_10_master_dict_LSTM_final.pkl'

for task in dep_vars_ls:
    # Run nested CV for the task
    outer_runs_dict, inner_runs_dict, best_model = CV_pipeline(None, volume_df, sMRI_ls, n_ses,
                                                               n_test, n_outer, n_inner, transformation, task, 'LSTM')
    
    # Save the outer runs dictionary
    with open(f'{filepath}/All3_10_outer_dict_{task}_LSTM_final.pkl', 'wb') as file:
        pickle.dump(outer_runs_dict, file)
    # Save the inner runs dictionary
    with open(f'{filepath}/All3_10_inner_dict_{task}_LSTM_final.pkl', 'wb') as file:
        pickle.dump(inner_runs_dict, file)
    # Save the best model
    with open(f'{filepath}/All3_10_best_outer_model_{task}_LSTM_final.pkl', 'wb') as file:
        pickle.dump(best_model, file)

    # Check if the file already exists
    if os.path.exists(master_path):
        # Load the existing dictionary
        with open(master_path, 'rb') as file:
            master_dict = pickle.load(file)

    else: # Otherwise create the dictionary
        master_dict = {}

    # Add entries
    master_dict[task] = [outer_runs_dict, inner_runs_dict, best_model]

    # Save the updated dictionary
    with open(master_path, 'wb') as file:
            pickle.dump(master_dict, file)

    print(f'Run for task {task} is completed')